## Pipeline to find interconnected regions
This pipeline takes in an <a href="https://github.com/danielsday/origami/"> Origami </a> processed DNA-DNA interactions generated using [fq2ChIAInts](fq2ChIAInts.ipynb) and finds interconnected networks of nodes (communities) using default or picked resolution. Plotting functions are also included to visualize output  
Range for resolution is from [0,1] with 1e-4 being something close to Insulated Neighborhoods, higher resolution makes bigger communities. You can also pick the "level", we start at level 0 then merging level 0 communities we get level 1 communities, so on and so forth.

In [11]:
# take a peek to see what DNA-DNA interactions are available
!ls /output_dir/*.bedpe

/output_dir/hicchip_mm1s_h3k27ac.bedpe
/output_dir/hichip25kb-filtered.bedpe
/output_dir/mES-SMC1-merged-petcount-filtered.bedpe


In [1]:
#Input DNA-DNA interactions and where should output text file of communities go?
dna_ints = "/output_dir/mES-SMC1-merged-petcount-filtered.bedpe"
out_comms = "/output_dir/mES-SMC1_chiapet_commmunities.txt"
resolution = 1
pick_level = 0

In [2]:
#import needed python packages
import networkx as nx, community, re, numpy as np, pandas as pd
from collections import Counter
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
#Function to plot communities
def plot_comm(G):
    axis=dict(showbackground=False,
              showline=False,
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title="")
    layout = go.Layout(
             title=G.graph["bounds"],
             width=600,
             height=600,
             showlegend=False,
             xaxis=go.XAxis(axis),
             yaxis=go.YAxis(axis),
             scene=go.Scene(
             xaxis=go.XAxis(axis),
             yaxis=go.YAxis(axis),
             zaxis=go.ZAxis(axis)),
        margin=go.Margin(l=0,
                         r=0,
                         b=0,
                         t=50),
        hovermode="closest")
    all_position = nx.spring_layout(G,dim=2)  
    #make node trace
    traceN = go.Scatter(x=[], y=[], mode="markers", text=[],marker=go.Marker(color=[],size=[],opacity=[]))
    traceN["name"] = ""
    traceN["hoverinfo"] = "text"
    for all_node in G.nodes(data=True):
        text_node = all_node[0]
        traceN["text"].append(text_node.replace("\n","<br>"))
        traceN["marker"]["color"].append("grey")
        traceN["marker"]["size"].append(20)
        traceN["marker"]["opacity"].append(1)
        traceN["x"].append(all_position[all_node[0]][0])
        traceN["y"].append(all_position[all_node[0]][1])

    traceE = go.Scatter(x=[], y=[], mode="lines", hoverinfo = "none")
    traceE["name"] = ""
    traceE["line"]["width"] = 1
    for edge in G.edges(data=True):
        traceE["x"] += [all_position[edge[0]][0],all_position[edge[1]][0], None]
        traceE["y"] += [all_position[edge[0]][1],all_position[edge[1]][1], None]

    data = go.Data([traceE, traceN])
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

## Louvain community finding
This chunk runs a <a href="https://en.wikipedia.org/wiki/Louvain_Modularity"> Louvain </a>  algorithm to partition the DNA-DNA interaction network into Communities. Output text file has the following format:
* First column is the start and end of the max / min of the community (linearly)
* Second column is the number of nodes that make up that community
* Third column and more are the member nodes

In [3]:
#Make a graph to store DNA-DNA interactions
dna_int_graph = nx.Graph(style="filled")
#Load DNA interactions
with open(dna_ints) as dna_ints_iter:
    for dna_int in dna_ints_iter:
        arr = dna_int.split()
        x = str(arr[0]) + ":" + str(arr[1]) + "-" + str(arr[2])
        mid_x = (int(arr[1])+int(arr[2]))/2.0
        y = str(arr[3]) + ":" + str(arr[4]) + "-" + str(arr[5])
        mid_y = (int(arr[4])+int(arr[5]))/2.0
        if (arr[0] == arr[3]):
            dna_int_graph.add_edge(x,y,label=1, capacity = 1, weight=float(arr[6]))

#Run louvain algorithm at defined resolution
dendogram_com = community.generate_dendrogram(dna_int_graph, resolution = resolution)
#Pull the first level of partitions
dna_int_comm = community.partition_at_level(dendogram_com, pick_level)
size2comm = Counter(dna_int_comm.values())
size_comms = [size for size in size2comm.values()]
boundary2nodes = dict()
bound_list_num_nodes = list()
bound_list_names = list()
comm_num_nodes = list()
#iterate over dna_int_comm keys and add each start/end to set stored in dict of comm --> min and comm-->max
inv_map = {}
for k, v in dna_int_comm.iteritems():
    inv_map[v] = inv_map.get(v, [])
    inv_map[v].append(k)
for comm, nodes in inv_map.iteritems():
    #get max and min of neighborhoods
    ref_node_arr = re.split(r"[-:]",nodes[0])
    min_comm = 1e10
    max_comm = 0
    for node in nodes:
        node_arr = re.split(r"[-:]",node)
        if node_arr[0] == ref_node_arr[0]:
            if (min(int(node_arr[1]),int(node_arr[2])) <= min_comm):
                min_comm = min(int(node_arr[1]),int(node_arr[2]))
            if (max(int(node_arr[1]),int(node_arr[2])) >= max_comm):
                max_comm = max(int(node_arr[1]),int(node_arr[2]))
    comm_name = ref_node_arr[0] + ":" + str(min_comm) + "-" + str(max_comm+1)
    boundary2nodes[comm_name] = nodes
    bound_list_num_nodes.append(len(nodes))
    bound_list_names.append(comm_name)
#Output text file with first column as the boundary of community, second # nodes, third all nodes w/ tabs
with open(out_comms, "w") as out_txt_comm: 
    for key, value in boundary2nodes.iteritems():
        out_txt_comm.write(key+"\t" + str(len(value)) + "\t")
        for node in value:
            out_txt_comm.write(node+"\t")
        out_txt_comm.write("\n")

# Find all Communities encompassing a DNA position
This allows you to pick a community to view as defined by a DNA position (ex. chr5:142275000), it will find ALL communities which encompass that location, print them, and plot the last one.

In [4]:
#Where do you want to look?
dna_position = "chr1:13117222"

In [5]:
#overlap with boundaries of defined communities and if it overlaps plot it
pick_split = re.split(r"[-:]",dna_position)
for key, value in boundary2nodes.iteritems():
    bound_split = re.split(r"[-:]",key)
    if (pick_split[0] == bound_split[0]) and (int(bound_split[1])<int(pick_split[1])) and (int(bound_split[2])>int(pick_split[1])): 
        print("target: " + dna_position + "\tCommunity Overlapping: " 
              + key + "\tlinear_length: " + 
              str((int(bound_split[2])-int(bound_split[1]))/1000) + "Kb" +
              "\tnum_nodes: " + str(len(value)))
        pick_comm_graph = dna_int_graph.subgraph(value)
        pick_comm_graph.graph["bounds"] = key
plot_comm(pick_comm_graph)

target: chr1:13117222	Community Overlapping: chr1:10824110-192754145	linear_length: 181930Kb	num_nodes: 6
target: chr1:13117222	Community Overlapping: chr1:9786216-178631988	linear_length: 168845Kb	num_nodes: 9
target: chr1:13117222	Community Overlapping: chr1:10128543-163851076	linear_length: 153722Kb	num_nodes: 14
target: chr1:13117222	Community Overlapping: chr1:4285523-161745045	linear_length: 157459Kb	num_nodes: 34
target: chr1:13117222	Community Overlapping: chr1:11632465-179650735	linear_length: 168018Kb	num_nodes: 19
target: chr1:13117222	Community Overlapping: chr1:3000949-44626647	linear_length: 41625Kb	num_nodes: 7
target: chr1:13117222	Community Overlapping: chr1:4902773-181394067	linear_length: 176491Kb	num_nodes: 3
target: chr1:13117222	Community Overlapping: chr1:10981575-174582775	linear_length: 163601Kb	num_nodes: 32
target: chr1:13117222	Community Overlapping: chr1:3471066-40827588	linear_length: 37356Kb	num_nodes: 23
target: chr1:13117222	Community Overlapping: chr1:

# Plot specific community as defined by name of boundaries

In [6]:
comm_name = "chr1:7076458-31212982"
pick_comm_graph = dna_int_graph.subgraph(boundary2nodes[comm_name])
pick_comm_graph.graph["bounds"] = comm_name
plot_comm(pick_comm_graph)

# Print top 10 communities with most nodes and plot #1

In [10]:
comm_sort_ix = np.argsort(bound_list_num_nodes)[::-1]
top_10_comm = [bound_list_names[comm] for comm in comm_sort_ix[0:9]]
for comm in top_10_comm:
    cur_nodes = boundary2nodes[comm]
    bound_split = re.split(r"[-:]",comm)
    print("Community name: " 
              + comm + "\tlinear_length: " + 
              str((int(bound_split[2])-int(bound_split[1]))/1000) + "Kb" +
              "\tnum_nodes: " + str(len(cur_nodes)))

comm_top_graph = dna_int_graph.subgraph(boundary2nodes[top_10_comm[0]])
comm_top_graph.graph["bounds"] = top_10_comm[0]
plot_comm(comm_top_graph)

Community name: chr8:27856320-128396908	linear_length: 100540Kb	num_nodes: 52
Community name: chr16:20225917-48995735	linear_length: 28769Kb	num_nodes: 51
Community name: chr4:91248003-140132276	linear_length: 48884Kb	num_nodes: 48
Community name: chr5:64678707-133497028	linear_length: 68818Kb	num_nodes: 48
Community name: chr4:22679722-147244697	linear_length: 124564Kb	num_nodes: 46
Community name: chr8:11941039-71059240	linear_length: 59118Kb	num_nodes: 46
Community name: chr3:6807431-153577822	linear_length: 146770Kb	num_nodes: 46
Community name: chr11:56780289-104783426	linear_length: 48003Kb	num_nodes: 45
Community name: chr8:14269605-95017485	linear_length: 80747Kb	num_nodes: 45
